<a href="https://colab.research.google.com/github/ACfma/4Work/blob/main/Copia_di_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
#splitting dataset in 80% train and 20% test
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
#Popping column to define objectives
train_labels, test_labels = train_dataset.pop('expenses'), test_dataset.pop('expenses')

In [ ]:
cat = ['sex', 'smoker', 'region']
num = ['age', 'bmi']

def df_to_ints(dataset, cat_column):
  '''
  Return input dataset in a fittable version for tensorflow.
  input: dataset: pd.Dataframe
         The input dataset
         cat_column: list or np.array
         the categorical column to be codified to int
  output: list of feature and dataset data as int
  '''
  for c in cat_column:
    tmp = dataset[c].unique()
    dic = {}
    for i,k in enumerate(tmp):
      dic[k] = i
    dataset[c] = dataset[c].map(dic)
  return dataset
train_dataset = df_to_ints(train_dataset, cat)
test_dataset  = df_to_ints(test_dataset, cat)

In [ ]:
test_dataset

In [ ]:
for k in train_dataset.columns.values:
  def z_sc(arr):
    m,s = arr.mean(), arr.std()
    arr = (arr-m)/s
    return arr
  train_dataset[k] = z_sc(train_dataset[k])
  test_dataset[k] = z_sc(test_dataset[k]) 

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(train_dataset)
model = tf.keras.Sequential([
    layers.Dense(units=1, kernel_initializer='he_uniform', use_bias=True)
])
model.compile(optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.1, momentum = 0.8),
    loss='mean_squared_error', metrics = [tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanSquaredError()])
BATCH_SIZE = 128
history = model.fit(np.asarray(train_dataset).astype('float32'), train_labels, batch_size = BATCH_SIZE, epochs = 30, steps_per_epoch = len(train_dataset.index)//BATCH_SIZE)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
